In [104]:
# Load the SQL extension to enable running SQL queries directly within the notebook
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [106]:
# # Connect to the Chinook SQLite database to run SQL queries within the notebook
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [3]:
# Import libraries for data handling, modeling, and evaluation:
# - pandas (pd) and numpy (np) for data manipulation and numerical operations
# - train_test_split for splitting data into training and test sets
# - StandardScaler for feature scaling to normalize inputs
# - LinearRegression, RandomForestRegressor, GradientBoostingRegressor for building predictive models
# - r2_score and mean_squared_error for assessing model performance

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

### 🎯 <span style="color: blue;">SQL Feature Engineering for CLTV Prediction</span>

This SQL pipeline creates customer-level features to power CLTV modeling:

1. **GenreStats**: Determines the top genre for each customer by counting how often they purchase each genre.
2. **TopGenres**: Extracts the top genre per customer using the `ROW_NUMBER()` window function.
3. **GenreDiversity**: Calculates how many distinct genres each customer has purchased.
4. **Final Aggregation**: Combines:
   - Customer's **top genre**
   - **Genre diversity**
   - **First and last purchase dates**
   - **Tenure** (customer lifespan)
   - **Recency** (days since last purchase)
   - **Purchase frequency** (invoices per month)
   - Extracted **email domain**
   - Total **Customer Lifetime Value (CLTV)**

These features are key inputs for training predictive models.


In [4]:
%%sql
WITH GenreStats AS (
    SELECT 
        i.CustomerId,
        g.Name AS TopGenre,
        COUNT(*) AS GenreCount,
        ROW_NUMBER() OVER (PARTITION BY i.CustomerId ORDER BY COUNT(*) DESC) AS GenreRank
    FROM Invoices i
    JOIN Invoice_items il ON i.InvoiceId = il.InvoiceId
    JOIN Tracks t ON il.TrackId = t.TrackId
    JOIN Genres g ON t.GenreId = g.GenreId
    GROUP BY i.CustomerId, g.Name
),
TopGenres AS (
    SELECT CustomerId, TopGenre
    FROM GenreStats
    WHERE GenreRank = 1
),
GenreDiversity AS (
    SELECT 
        i.CustomerId,
        COUNT(DISTINCT g.GenreId) AS NumGenresPurchased
    FROM Invoices i
    JOIN Invoice_items ii ON i.InvoiceId = ii.InvoiceId
    JOIN Tracks t ON ii.TrackId = t.TrackId
    JOIN Genres g ON t.GenreId = g.GenreId
    GROUP BY i.CustomerId
)

SELECT 
    c.CustomerId,
    c.Country,
    tg.TopGenre,
    gd.NumGenresPurchased,
    MIN(i.InvoiceDate) AS FirstPurchase,
    MAX(i.InvoiceDate) AS LastPurchase,
    JULIANDAY(MAX(i.InvoiceDate)) - JULIANDAY(MIN(i.InvoiceDate)) AS Tenure,
    JULIANDAY('2013-12-31') - JULIANDAY(MAX(i.InvoiceDate)) AS Recency,
    -- Frequency: invoices per month
    COUNT(i.InvoiceId) / ((JULIANDAY(MAX(i.InvoiceDate)) - JULIANDAY(MIN(i.InvoiceDate))) / 30.0) AS PurchaseFrequency,
    SUBSTR(c.Email, INSTR(c.Email, '@') + 1) AS EmailDomain,
    SUM(i.Total) AS TotalCLTV
FROM Customers c
LEFT JOIN Invoices i ON c.CustomerId = i.CustomerId
LEFT JOIN TopGenres tg ON c.CustomerId = tg.CustomerId
LEFT JOIN GenreDiversity gd ON c.CustomerId = gd.CustomerId
GROUP BY c.CustomerId, c.Country, tg.TopGenre, gd.NumGenresPurchased;


 * sqlite:///chinook.db
Done.


CustomerId,Country,TopGenre,NumGenresPurchased,FirstPurchase,LastPurchase,Tenure,Recency,PurchaseFrequency,EmailDomain,TotalCLTV
1,Brazil,Rock,8,2010-03-11 00:00:00,2013-08-07 00:00:00,1245.0,146.0,0.1686746987951807,embraer.com.br,39.62
2,Germany,Rock,7,2009-01-01 00:00:00,2012-07-13 00:00:00,1289.0,536.0,0.16291698991466252,surfeu.de,37.62
3,Canada,Metal,10,2010-03-11 00:00:00,2013-09-20 00:00:00,1289.0,102.0,0.16291698991466252,gmail.com,39.62
4,Norway,Rock,8,2009-01-02 00:00:00,2013-10-03 00:00:00,1735.0,89.0,0.12103746397694524,yahoo.no,39.62
5,Czech Republic,Rock,8,2009-12-08 00:00:00,2013-05-06 00:00:00,1245.0,239.0,0.1686746987951807,jetbrains.com,40.62
6,Czech Republic,Rock,9,2009-07-11 00:00:00,2013-11-13 00:00:00,1586.0,48.0,0.13240857503152584,gmail.com,49.62
7,Austria,Rock,9,2009-12-08 00:00:00,2013-06-19 00:00:00,1289.0,195.0,0.16291698991466252,apple.at,42.62
8,Belgium,Rock,4,2009-01-03 00:00:00,2013-10-04 00:00:00,1735.0,88.0,0.12103746397694524,apple.be,37.62
9,Denmark,Rock,5,2009-09-06 00:00:00,2013-02-02 00:00:00,1245.0,332.0,0.1686746987951807,jubii.dk,37.62
10,Brazil,Rock,7,2009-04-09 00:00:00,2013-08-12 00:00:00,1586.0,141.0,0.13240857503152584,woodstock.com.br,37.62


In [5]:
# Save the result and convert it into a Pandas DataFrame for easier analysis
df_result = _ 
df_df = df_result.DataFrame()

In [6]:
# # Display the first few rows of the DataFrame to inspect the structure and content
df_df.head()

,CustomerId,Country,TopGenre,NumGenresPurchased,FirstPurchase,LastPurchase,Tenure,Recency,PurchaseFrequency,EmailDomain,TotalCLTV
0,1,Brazil,Rock,8,2010-03-11 00:00:00,2013-08-07 00:00:00,1245.0,146.0,0.168675,embraer.com.br,39.62
1,2,Germany,Rock,7,2009-01-01 00:00:00,2012-07-13 00:00:00,1289.0,536.0,0.162917,surfeu.de,37.62
2,3,Canada,Metal,10,2010-03-11 00:00:00,2013-09-20 00:00:00,1289.0,102.0,0.162917,gmail.com,39.62
3,4,Norway,Rock,8,2009-01-02 00:00:00,2013-10-03 00:00:00,1735.0,89.0,0.121037,yahoo.no,39.62
4,5,Czech Republic,Rock,8,2009-12-08 00:00:00,2013-05-06 00:00:00,1245.0,239.0,0.168675,jetbrains.com,40.62


In [7]:
# Calculate the skewness of the TotalCLTV column to understand the distribution shape.
# Skewness > 1 indicates strong right skew (long tail to the right), which can hurt model performance.
# Helps determine if a log transformation is necessary.
skewness = df_df["TotalCLTV"].skew()
print(f"Skewness of TotalCLTV: {skewness:.3f}")

Skewness of TotalCLTV: 1.806


In [8]:
# Log-transform the CLTV to reduce right skewness and normalize the distribution.
# This helps regression models perform better, especially when there are extreme high values (outliers).
# We use log1p to safely handle zero values (log1p(x) = log(1 + x)).
df_df['Log_CLTV'] = np.log1p(df_df['TotalCLTV'])

In [93]:
# Display the first few rows
df_df.head()

,CustomerId,Country,TopGenre,NumGenresPurchased,FirstPurchase,LastPurchase,Tenure,Recency,PurchaseFrequency,EmailDomain,TotalCLTV,Log_CLTV
0,1,Brazil,Rock,8,2010-03-11 00:00:00,2013-08-07 00:00:00,1245.0,146.0,0.168675,embraer.com.br,39.62,3.704261
1,2,Germany,Rock,7,2009-01-01 00:00:00,2012-07-13 00:00:00,1289.0,536.0,0.162917,surfeu.de,37.62,3.653770
2,3,Canada,Metal,10,2010-03-11 00:00:00,2013-09-20 00:00:00,1289.0,102.0,0.162917,gmail.com,39.62,3.704261
3,4,Norway,Rock,8,2009-01-02 00:00:00,2013-10-03 00:00:00,1735.0,89.0,0.121037,yahoo.no,39.62,3.704261
4,5,Czech Republic,Rock,8,2009-12-08 00:00:00,2013-05-06 00:00:00,1245.0,239.0,0.168675,jetbrains.com,40.62,3.728581


In [10]:
# Drop CustomerId (identifier), First/LastPurchase (used to calculate DaysActive),
# and TotalCLTV because we already created a normalized version (Log_CLTV) for modeling.
# Keeping TotalCLTV could cause data leakage or reduce model accuracy.
df_model = df_df.drop(columns=['CustomerId', 'FirstPurchase', 'LastPurchase', 'TotalCLTV'])

In [94]:
# Display the first few rows
df_model.head()

,Country,TopGenre,NumGenresPurchased,Tenure,Recency,PurchaseFrequency,EmailDomain,Log_CLTV
0,Brazil,Rock,8,1245.0,146.0,0.168675,embraer.com.br,3.704261
1,Germany,Rock,7,1289.0,536.0,0.162917,surfeu.de,3.653770
2,Canada,Metal,10,1289.0,102.0,0.162917,gmail.com,3.704261
3,Norway,Rock,8,1735.0,89.0,0.121037,yahoo.no,3.704261
4,Czech Republic,Rock,8,1245.0,239.0,0.168675,jetbrains.com,3.728581


In [12]:
# Check for multicollinearity (on numerical data only)
correlation_matrix = df_model[['NumGenresPurchased', 'Tenure', 'Recency', 'PurchaseFrequency']].corr()
print("\nCorrelation Matrix (Numeric Features):")
print(correlation_matrix)


Correlation Matrix (Numeric Features):
                    NumGenresPurchased    Tenure   Recency  PurchaseFrequency
NumGenresPurchased            1.000000 -0.043204  0.034991           0.050198
Tenure                       -0.043204  1.000000 -0.579990          -0.987580
Recency                       0.034991 -0.579990  1.000000           0.545404
PurchaseFrequency             0.050198 -0.987580  0.545404           1.000000


In [95]:
# One of the features (either Tenure or PurchaseFrequency) will be dropped due to a high negative correlation between them (-0.987580),
# which can lead to multicollinearity and adversely affect the stability and interpretability of the regression model.

df_model_new = df_model.drop(columns=['PurchaseFrequency'])

In [14]:
# One-hot encode categorical features
df_encoded = pd.get_dummies(df_model_new, columns=['Country', 'TopGenre', 'EmailDomain'], drop_first=True)

In [96]:
# Display the first few rows
df_encoded.head()

,NumGenresPurchased,Tenure,Recency,Log_CLTV,Country_Australia,Country_Austria,Country_Belgium,Country_Brazil,Country_Canada,Country_Chile,...,EmailDomain_yahoo.de,EmailDomain_yahoo.es,EmailDomain_yahoo.fr,EmailDomain_yahoo.in,EmailDomain_yahoo.it,EmailDomain_yahoo.nl,EmailDomain_yahoo.no,EmailDomain_yahoo.pt,EmailDomain_yahoo.se,EmailDomain_yahoo.uk
0,8,1245.0,146.0,3.704261,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1289.0,536.0,3.653770,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,1289.0,102.0,3.704261,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,8,1735.0,89.0,3.704261,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8,1245.0,239.0,3.728581,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Define X and y
X = df_encoded.drop(columns=['Log_CLTV'])  # features
y = df_encoded['Log_CLTV']                # target

In [98]:
# Display the first few rows of X
X.head()

,NumGenresPurchased,Tenure,Recency,Country_Australia,Country_Austria,Country_Belgium,Country_Brazil,Country_Canada,Country_Chile,Country_Czech Republic,...,EmailDomain_yahoo.de,EmailDomain_yahoo.es,EmailDomain_yahoo.fr,EmailDomain_yahoo.in,EmailDomain_yahoo.it,EmailDomain_yahoo.nl,EmailDomain_yahoo.no,EmailDomain_yahoo.pt,EmailDomain_yahoo.se,EmailDomain_yahoo.uk
0,8,1245.0,146.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1289.0,536.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,1289.0,102.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,1735.0,89.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8,1245.0,239.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [99]:
# Display the first few rows of y
y.head()

0    3.704261
1    3.653770
2    3.704261
3    3.704261
4    3.728581
Name: Log_CLTV, dtype: float64

In [19]:
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### <span style="color: blue;">LINEAR REGRESSION MODEL</span>

In [21]:
# Initialize and Train the model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

LinearRegression()

In [22]:
# Predict using the trained model
lr_y_pred = lr_model.predict(X_test_scaled)

In [23]:
# Evaluate the model
mse_lr = mean_squared_error(y_test, lr_y_pred)
r2_lr = r2_score(y_test, lr_y_pred)

In [101]:
print ("mse_lr :", mse_lr)
print ("r2_lr :", r2_lr)

mse_lr : 0.010729495937269311
r2_lr : -0.2038952347963221


### <span style="color: blue;">RANDOM FOREST </span>

In [27]:
# Initialize and Train the model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [28]:
# Make predictions
rf_y_pred = rf_model.predict(X_test_scaled)

In [29]:
# Evaluate the model
mse_rf = mean_squared_error(y_test, rf_y_pred)
r2_rf = r2_score(y_test, rf_y_pred)

In [100]:
print ("mse_rf :", mse_rf)
print ("r2_rf :", r2_rf)

mse_rf : 0.01269459566393396
r2_rf : -0.42438781065102926


### <span style="color: blue;">GRADIENT BOOSTING</span> 

In [32]:
# Initialize and Train the model
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor(random_state=42)

In [33]:
# Make predictions
gb_y_pred = gb_model.predict(X_test_scaled)

In [34]:
# Evaluate the model
mse_gb = mean_squared_error(y_test, gb_y_pred)
r2_gb = r2_score(y_test, gb_y_pred)

In [102]:
print ("mse_gb:", mse_gb)
print ("r2_gb :", r2_gb)

mse_gb: 0.013118091399989403
r2_gb : -0.4719058396036053


#####  <span style="color: blue;">Among the three models, Linear Regression performed best with an R² of (<span style="color: red;">-0.2038952347963221</span>), closely followed by Random Forest (<span style="color: red;">-0.42438781065102926</span>), while Gradient boosting came last with an R² of (<span style="color: red;">-0.4719058396036053</span>). Nevertheless, a negative R² indicates that the model performs worse than simply predicting the mean of the target variable, signaling a poor fit and limited predictive power.</span>

####  <span style="color: blue;">Leveraging the feature importance from each of the models, I plan to rebuild the models with a refined feature set to reduce noise and improve performance — with the goal of achieving a positive R² score.</span>

#### <span style="color: blue;">Further Analysis for Improvement</span>

In [37]:
# EmailDomain Analysis
email_counts = df_model['EmailDomain'].value_counts()
print("\nEmailDomain Value Counts:")
print(email_counts)
print(f"Number of EmailDomain categories: {len(email_counts)}")
print(f"EmailDomains with count=1: {sum(email_counts == 1)}")


EmailDomain Value Counts:
gmail.com           8
hotmail.com         4
shaw.ca             3
yahoo.fr            2
surfeu.de           2
yahoo.com           2
uol.com.br          2
yahoo.de            2
aol.com             2
yahoo.it            1
apple.fr            1
apple.fi            1
apple.hu            1
apple.ie            1
embraer.com.br      1
yahoo.nl            1
yahoo.es            1
yahoo.se            1
yahoo.uk            1
yahoo.au            1
yahoo.ar            1
yahoo.cl            1
rediff.com          1
wp.pl               1
yachoo.ca           1
sapo.pt             1
yahoo.pt            1
yahoo.ca            1
comcast.com         1
apple.com           1
microsoft.com       1
google.com          1
rogers.ca           1
riotur.gov.br       1
woodstock.com.br    1
jubii.dk            1
apple.be            1
apple.at            1
jetbrains.com       1
yahoo.no            1
yahoo.in            1
Name: EmailDomain, dtype: int64
Number of EmailDomain categories: 41
Em

In [38]:
# Country Analysis 
country_counts = df_df['Country'].value_counts()
print("Country Value Counts:")
print(country_counts)
print(f"Total unique countries: {len(country_counts)}")
print(f"Countries with count=1: {sum(country_counts == 1)}")

Country Value Counts:
USA               13
Canada             8
Brazil             5
France             5
Germany            4
United Kingdom     3
Portugal           2
India              2
Czech Republic     2
Denmark            1
Belgium            1
Austria            1
Finland            1
Ireland            1
Italy              1
Netherlands        1
Poland             1
Spain              1
Sweden             1
Norway             1
Australia          1
Argentina          1
Chile              1
Hungary            1
Name: Country, dtype: int64
Total unique countries: 24
Countries with count=1: 15


In [39]:
# I noticed EmailDomain had 41 unique categories, with 32 appearing only once across 59 rows.
# These singletons, like yahoo.it, expand into sparse dummy variables (e.g., 58 zeros, 1 one), 
# contributing minimal signal and significant noise. 
# Group rare email domains (appearing <= 2 times) into 'Other' to reduce sparsity and prevent overfitting.

# This results in fewer categories, consolidating predictive power and improving model generalization.


threshold = 2  # Domains with >2 counts stay, others become 'Other'
email_counts = df_model_new['EmailDomain'].value_counts()
df_model_new['EmailDomain_grouped'] = df_model_new['EmailDomain'].apply(
    lambda x: x if email_counts[x] > threshold else 'Other'
)
df_model_adjusted = df_model_new.drop(columns=['EmailDomain'])


In [40]:
df_model_adjusted.head()

,Country,TopGenre,NumGenresPurchased,Tenure,Recency,Log_CLTV,EmailDomain_grouped
0,Brazil,Rock,8,1245.0,146.0,3.704261,Other
1,Germany,Rock,7,1289.0,536.0,3.653770,Other
2,Canada,Metal,10,1289.0,102.0,3.704261,gmail.com
3,Norway,Rock,8,1735.0,89.0,3.704261,Other
4,Czech Republic,Rock,8,1245.0,239.0,3.728581,Other


In [41]:
# Group Country
threshold = 2 # Contries with >2 counts stay, others become 'Other'
country_counts = df_model_adjusted['Country'].value_counts()
df_model_adjusted['Country_grouped'] = df_model_adjusted['Country'].apply(
    lambda x: x if country_counts[x] > 2 else 'Other'
)
df_model_adjusted = df_model_adjusted.drop(columns=['Country'])

In [42]:
df_model_adjusted.head()

,TopGenre,NumGenresPurchased,Tenure,Recency,Log_CLTV,EmailDomain_grouped,Country_grouped
0,Rock,8,1245.0,146.0,3.704261,Other,Brazil
1,Rock,7,1289.0,536.0,3.653770,Other,Germany
2,Metal,10,1289.0,102.0,3.704261,gmail.com,Canada
3,Rock,8,1735.0,89.0,3.704261,Other,Other
4,Rock,8,1245.0,239.0,3.728581,Other,Other


In [43]:
# Encode adjusted DataFrame
df_encoded_adjusted = pd.get_dummies(df_model_adjusted, columns=['Country_grouped', 'TopGenre', 'EmailDomain_grouped'], drop_first=True)

In [44]:
# Separate features and target variable for model training.
X_adjusted = df_encoded_adjusted.drop(columns=['Log_CLTV'])
y_adjusted = df_encoded_adjusted['Log_CLTV']

In [45]:
# Preview the first few rows of the feature set to verify data structure and contents.
X_adjusted.head()

,NumGenresPurchased,Tenure,Recency,Country_grouped_Canada,Country_grouped_France,Country_grouped_Germany,Country_grouped_Other,Country_grouped_USA,Country_grouped_United Kingdom,TopGenre_Metal,TopGenre_Rock,EmailDomain_grouped_gmail.com,EmailDomain_grouped_hotmail.com,EmailDomain_grouped_shaw.ca
0,8,1245.0,146.0,0,0,0,0,0,0,0,1,0,0,0
1,7,1289.0,536.0,0,0,1,0,0,0,0,1,0,0,0
2,10,1289.0,102.0,1,0,0,0,0,0,1,0,1,0,0
3,8,1735.0,89.0,0,0,0,1,0,0,0,1,0,0,0
4,8,1245.0,239.0,0,0,0,1,0,0,0,1,0,0,0


In [46]:
# Split the data into training and testing sets.
X_train_adjusted, X_test_adjusted, y_train_adjusted, y_test_adjusted = train_test_split(X_adjusted, y_adjusted, test_size=0.2, random_state=42)

In [47]:
# Standardize the feature data using StandardScaler.
X_train_adjusted_scaled = scaler.fit_transform(X_train_adjusted)
X_test_adjusted_scaled = scaler.transform(X_test_adjusted)

#### <span style="color: blue;">Linear Regression</span>

In [48]:
# Initialize and Train the model.
lr_model_adjusted = LinearRegression()
lr_model_adjusted.fit(X_train_adjusted_scaled, y_train_adjusted)

LinearRegression()

In [49]:
# Make Predictions
y_pred_lr_adjusted = lr_model_adjusted.predict(X_test_adjusted_scaled)

In [50]:
# Evaluate the model
mse_lr_adjusted = mean_squared_error(y_test_adjusted, y_pred_lr_adjusted)
r2_lr_adjusted = r2_score(y_test_adjusted, y_pred_lr_adjusted)

In [51]:
print ("mse_lr_adjusted :", mse_lr_adjusted)
print ("r2_lr_adjusted :", r2_lr_adjusted)

mse_lr_adjusted : 0.01042269411951826
r2_lr_adjusted : -0.16947076149610263


#### <span style="color: blue;">Random Forest</span>

In [52]:
# Initialize and Train the model
rf_model_adjusted = RandomForestRegressor(random_state=42)
rf_model_adjusted.fit(X_train_adjusted_scaled, y_train_adjusted)

RandomForestRegressor(random_state=42)

In [53]:
# Make Predictions
y_pred_rf_adjusted = rf_model_adjusted.predict(X_test_adjusted_scaled)

In [54]:
# Evaluate the model
mse_rf_adjusted = mean_squared_error(y_test_adjusted, y_pred_rf_adjusted)
r2_rf_adjusted = r2_score(y_test_adjusted, y_pred_rf_adjusted)

In [55]:
print ("mse_rf_adjusted: ", mse_rf_adjusted)
print ("r_rf_adjusted: ", r2_rf_adjusted)

mse_rf_adjusted:  0.011313685845048375
r_rf_adjusted:  -0.26944383561627006


#### <span style="color: blue;">Gradient Boosting</span>

In [56]:
# Initialize and Train the model
gb_model_adjusted = GradientBoostingRegressor(random_state=42)
gb_model_adjusted.fit(X_train_adjusted_scaled, y_train_adjusted)

GradientBoostingRegressor(random_state=42)

In [57]:
# Make predictions
y_pred_gb_adjusted = gb_model_adjusted.predict(X_test_adjusted_scaled)

In [58]:
# Evaluate the model 
mse_gb_adjusted = mean_squared_error(y_test_adjusted, y_pred_gb_adjusted)
r2_gb_adjusted = r2_score(y_test_adjusted, y_pred_gb_adjusted)

In [59]:
print("mse_gb_adjusted: ", mse_gb_adjusted)
print("r2_gb_adjusted: ", r2_gb_adjusted)

mse_gb_adjusted:  0.013248886778545329
r2_gb_adjusted:  -0.48658163927746534


### <span style="color: blue;">Although the initial model yielded an R² of <span style="color: red;">-0.2039</span>, subsequent feature refinement and tuning improved the model’s performance, resulting in a better R² of <span style="color: red;">-0.1695</span> and demonstrating progress toward a more reliable predictive fit.</span>

#### <span style="color: blue;">Linear Regression with top features</span>

In [60]:
# LInear regression feature importance 
feature_importance = pd.DataFrame({
    'Feature': X_adjusted.columns,
    'Coefficient': lr_model_adjusted.coef_,
    'Absolute_Coefficient': abs(lr_model_adjusted.coef_)
}).sort_values(by='Absolute_Coefficient', ascending=False)

In [61]:
print("\nTop 20 Features (Linear Regression Absolute Coefficients):")
print(feature_importance.head(20))


Top 20 Features (Linear Regression Absolute Coefficients):
                            Feature  Coefficient  Absolute_Coefficient
0                NumGenresPurchased     0.037661              0.037661
2                           Recency    -0.016709              0.016709
6             Country_grouped_Other     0.015603              0.015603
12  EmailDomain_grouped_hotmail.com    -0.013478              0.013478
11    EmailDomain_grouped_gmail.com     0.013384              0.013384
5           Country_grouped_Germany     0.009827              0.009827
8    Country_grouped_United Kingdom     0.006199              0.006199
9                    TopGenre_Metal    -0.005865              0.005865
3            Country_grouped_Canada    -0.004753              0.004753
13      EmailDomain_grouped_shaw.ca     0.004381              0.004381
10                    TopGenre_Rock     0.002510              0.002510
4            Country_grouped_France     0.002169              0.002169
7               C

In [62]:
# Select Top Features (Top 15)
top_features = feature_importance['Feature'].head(15).tolist()

In [63]:
# Show Linear Regression top features
print("\nSelected Top 15 Features:", top_features)


Selected Top 15 Features: ['NumGenresPurchased', 'Recency', 'Country_grouped_Other', 'EmailDomain_grouped_hotmail.com', 'EmailDomain_grouped_gmail.com', 'Country_grouped_Germany', 'Country_grouped_United Kingdom', 'TopGenre_Metal', 'Country_grouped_Canada', 'EmailDomain_grouped_shaw.ca', 'TopGenre_Rock', 'Country_grouped_France', 'Country_grouped_USA', 'Tenure']


In [64]:
# Select the top features based on feature importance for Linear Regression.
X_selected_lr = X_adjusted[top_features]
y_selected_lr = y_adjusted

In [65]:
# list few roles 
X_selected_lr.head()

,NumGenresPurchased,Recency,Country_grouped_Other,EmailDomain_grouped_hotmail.com,EmailDomain_grouped_gmail.com,Country_grouped_Germany,Country_grouped_United Kingdom,TopGenre_Metal,Country_grouped_Canada,EmailDomain_grouped_shaw.ca,TopGenre_Rock,Country_grouped_France,Country_grouped_USA,Tenure
0,8,146.0,0,0,0,0,0,0,0,0,1,0,0,1245.0
1,7,536.0,0,0,0,1,0,0,0,0,1,0,0,1289.0
2,10,102.0,0,0,1,0,0,1,1,0,0,0,0,1289.0
3,8,89.0,1,0,0,0,0,0,0,0,1,0,0,1735.0
4,8,239.0,1,0,0,0,0,0,0,0,1,0,0,1245.0


In [66]:
# Split the data into training and testing sets after selecting the top features.
# Use a test size of 0.1 due to the smaller dataset.
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected_lr, y_selected_lr, test_size=0.1, random_state=42)

In [67]:
# Scale
scaler_selected = StandardScaler()
X_train_selected_scaled_lr = scaler.fit_transform(X_train_selected)
X_test_selected_scaled_lr = scaler.transform(X_test_selected)

In [68]:
# Initialize and Train the model
lr_model_selected = LinearRegression()
lr_model_selected.fit(X_train_selected_scaled_lr, y_train_selected)

LinearRegression()

In [69]:
# Make Predictions
y_pred_lr_selected = lr_model_selected.predict(X_test_selected_scaled_lr)

In [70]:
# Evaluate the Model
mse_lr_selected = mean_squared_error(y_test_selected, y_pred_lr_selected)
r2_lr_selected = r2_score(y_test_selected, y_pred_lr_selected)

In [71]:
# Print the MSE and R² values to evaluate the performance of the Linear Regression model with selected features.
print("mse_lr_selected: ", mse_lr_selected)
print("r2_lr_selected: ", r2_lr_selected)

mse_lr_selected:  0.007794814780897199
r2_lr_selected:  0.3118879758418829


#### <span style="color: blue;">Random forest with top features </span> 

In [72]:
# Get feature importances
rf_importances = pd.DataFrame({
    'Feature': X_adjusted.columns,
    'Importance': rf_model_adjusted.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [73]:
# Select top 20
rf_top_features = rf_importances.head(20)['Feature'].tolist()

In [74]:
# Show Random Forest top features
print("\nSelected Top 20 Features:", rf_top_features)


Selected Top 20 Features: ['NumGenresPurchased', 'Recency', 'Tenure', 'Country_grouped_USA', 'EmailDomain_grouped_gmail.com', 'Country_grouped_Other', 'TopGenre_Metal', 'Country_grouped_France', 'EmailDomain_grouped_hotmail.com', 'TopGenre_Rock', 'Country_grouped_Canada', 'Country_grouped_Germany', 'EmailDomain_grouped_shaw.ca', 'Country_grouped_United Kingdom']


In [75]:
# Select the top features based on feature importance for Random forest
X_selected_rf = X_adjusted[rf_top_features]
y_selected_rf = y_adjusted

In [76]:
# Show few roles
X_selected_rf.head()

,NumGenresPurchased,Recency,Tenure,Country_grouped_USA,EmailDomain_grouped_gmail.com,Country_grouped_Other,TopGenre_Metal,Country_grouped_France,EmailDomain_grouped_hotmail.com,TopGenre_Rock,Country_grouped_Canada,Country_grouped_Germany,EmailDomain_grouped_shaw.ca,Country_grouped_United Kingdom
0,8,146.0,1245.0,0,0,0,0,0,0,1,0,0,0,0
1,7,536.0,1289.0,0,0,0,0,0,0,1,0,1,0,0
2,10,102.0,1289.0,0,1,0,1,0,0,0,1,0,0,0
3,8,89.0,1735.0,0,0,1,0,0,0,1,0,0,0,0
4,8,239.0,1245.0,0,0,1,0,0,0,1,0,0,0,0


In [77]:
# Split the data into training and testing sets after selecting the top features.
# Use a test size of 0.1 due to the smaller dataset.
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected_rf, y_selected_rf, test_size=0.1, random_state=42)

In [78]:
# Initialize and Train the model
rf_model_selected = RandomForestRegressor(random_state=42)
rf_model_selected.fit(X_train_selected, y_train_selected)

RandomForestRegressor(random_state=42)

In [79]:
# Make Predictions
y_pred_rf_selected = rf_model_selected.predict(X_test_selected)

In [80]:
# Evaluate the Model
mse_rf_selected = mean_squared_error(y_test_selected, y_pred_rf_selected)
r2_rf_selected = r2_score(y_test_selected, y_pred_rf_selected)

In [81]:
# Print the MSE and R² values to evaluate the performance of the Random Forest model with selected features.
print("mse_rf_selected: ", mse_rf_selected)
print("r2_rf_selected: ", r2_rf_selected)

mse_rf_selected:  0.014511352820280694
r2_rf_selected:  -0.28103574531460573


#### <span style="color: blue;">Gradient Boosting with top features </span>

In [82]:
# Get feature importances
gb_importances = pd.DataFrame({
    'Feature': X_adjusted.columns,
    'Importance': gb_model_adjusted.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Select top 20 features
gb_top_features = gb_importances.head(20)['Feature'].tolist()

In [83]:
# Show Gradient Boosting top features
print("\nSelected Top 20 Features:", gb_top_features)


Selected Top 20 Features: ['NumGenresPurchased', 'Recency', 'Tenure', 'EmailDomain_grouped_gmail.com', 'Country_grouped_USA', 'Country_grouped_Canada', 'TopGenre_Rock', 'Country_grouped_Other', 'Country_grouped_France', 'TopGenre_Metal', 'EmailDomain_grouped_hotmail.com', 'Country_grouped_Germany', 'EmailDomain_grouped_shaw.ca', 'Country_grouped_United Kingdom']


In [84]:
# Select the top features based on feature importance for Gradient Boosting
X_selected_gb = X_adjusted[gb_top_features]
y_selected_gb = y_adjusted

In [85]:
# List first few rows
X_selected_gb.head()

,NumGenresPurchased,Recency,Tenure,EmailDomain_grouped_gmail.com,Country_grouped_USA,Country_grouped_Canada,TopGenre_Rock,Country_grouped_Other,Country_grouped_France,TopGenre_Metal,EmailDomain_grouped_hotmail.com,Country_grouped_Germany,EmailDomain_grouped_shaw.ca,Country_grouped_United Kingdom
0,8,146.0,1245.0,0,0,0,1,0,0,0,0,0,0,0
1,7,536.0,1289.0,0,0,0,1,0,0,0,0,1,0,0
2,10,102.0,1289.0,1,0,1,0,0,0,1,0,0,0,0
3,8,89.0,1735.0,0,0,0,1,1,0,0,0,0,0,0
4,8,239.0,1245.0,0,0,0,1,1,0,0,0,0,0,0


In [86]:
# Split the data into training and testing sets after selecting the top features.
# Use a test size of 0.1 due to the smaller dataset.
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected_gb, y_selected_gb, test_size=0.1, random_state=42)

In [87]:
# Initialize and Train the model
gb_model_selected = GradientBoostingRegressor(random_state=42)
gb_model_selected.fit(X_train_selected, y_train_selected)

GradientBoostingRegressor(random_state=42)

In [88]:
# Make Predictions
y_pred_gb_selected = gb_model_selected.predict(X_test_selected)

In [89]:
# Evaluate the Model
mse_gb_selected = mean_squared_error(y_test_selected, y_pred_gb_selected)
r2_gb_selected = r2_score(y_test_selected, y_pred_gb_selected)

In [90]:
# Print the MSE and R² values to evaluate the performance of the Gradient Boosting model with selected features.
print("mse_gb_selected: ", mse_gb_selected)
print("r2_gb_selected: ", r2_gb_selected)

mse_gb_selected:  0.016848137924413045
r2_gb_selected:  -0.48732286992566287


#### <span style="color: blue;">Despite the limited nature of the Chinook dataset, careful preprocessing, transformation, and feature selection allowed Linear Regression to emerge as the best-performing model for CLTV prediction. While the R² of 0.31 suggests moderate explanatory power, the model demonstrates that customer behavior and demographics can offer meaningful insights into their overall lifetime value.</span>

#### <span style="color: blue;">Save the best model(Linear regression) and the scaler</span>

In [91]:
import joblib

# Saving the model and scaler, Linear Regression in this case (optional, not needed for this static project)
joblib.dump(lr_model_selected, 'models/clv_lr_top_model.pkl')

# Save the scaler used for the selected features
joblib.dump(scaler_selected, 'models/scaler_top_features.pkl')

# To load the model later:
# lr_model_loaded = joblib.load('models/lr_model_selected.pkl')





['models/scaler_top_features.pkl']